#Fast Food SVM
Application of the Fast Food kernel expansion algorithm to SVMs.

In [49]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pylab as pl
import theano
import theano.tensor as T
import numpy as np
from numpy.linalg import norm
import pandas as pd
import sklearn.datasets
from sklearn import svm
from scipy.linalg import hadamard
from scipy.special import gammaincinv
from functools import partial

VERBOSE = True  # increases verbosity of some outputs

# random number generator
rng = np.random.RandomState()

# load data
iris = sklearn.datasets.load_iris()
X = iris.data.T # each column is an input pattern
y = iris.target
print('Shape of X:',X.shape,'; shape of y:',y.shape)

# pad with zeros so d is nearest power of 2
# N = X.shape[0]
# l = X.shape[1]
# print('Original shape of X: n =',N,', d =',l)
# d = int(2 ** np.ceil(np.log2(l)))
# if d != l:  # pad only if needed
#     print('Padding input from d =',l,'to d =',d)
#     X = np.pad(X,((0,d-l),(0,0)),mode='constant',constant_values=0)
    
# convert to shared tensors
#X = theano.shared(X,name='X',borrow=True)
#y = theano.shared(y,name='y',borrow=True)

Shape of X: (4, 150) ; shape of y: (150,)


In [52]:
def fastfood_params(n,d):
    l = int(np.ceil(np.log2(d)))
    d = 2**l
    k = int(np.ceil(n/d))
    n = d*k

    # M = np.diag(v) on each vector v to diagonalize it into the matrix M
    
    
    B = []
    G = []
    PI = []
    S = []
    for ii in xrange(k):
        B_ii  = rng.choice([-1,1],size=k)
        G_ii  = rng.normal(size=k)
        PI_ii = rng.permutation(d)
        
        B.append(B_ii)
        G.append(G_ii)
        PI.append(PI_ii)
        
        p1 = rng.uniform(size=d)
        p2 = d/2
        print('p1 =',p1,'; p2 =',p2)
        T = gammaincinv(p1,p2)
        print('T1 =',T)
        T = (T*2) ** (1/2)
        print('T2 =',T)
        s_i = T * norm(G,'fro')**(-1)
        print('s_i =', s_i)
        S_ii = s_i
        S.append(S_ii)
    
    B = np.vstack(B)
    G = np.vstack(G)
    PI = np.vstack(PI)
    S = np.vstack(S)
    return B, G, PI, S
print('Ready to generate fastfood params')

Ready to generate fastfood params


In [53]:
d = 100 # dimension of input pattern
n = d*20 # basis number used for approximation
sgm = 10 # bandwidth for Gaussian kernel

params = fastfood_params(n,d)
print('Fastfood params:',params)

p1 = [ 0.01207189  0.9471411   0.84721235  0.3894581   0.84761039  0.68176524
  0.48537136  0.589344    0.55122684  0.29782981  0.86927135  0.03199064
  0.08521363  0.66583595  0.72648202  0.25895693  0.26376139  0.27720875
  0.58984485  0.20876025  0.18423904  0.7194444   0.13017745  0.56574079
  0.52640152  0.58470017  0.39437681  0.48608996  0.53265572  0.77394882
  0.16299292  0.1851869   0.30134096  0.94730668  0.11092535  0.13878537
  0.48646079  0.93097098  0.33981988  0.73292744  0.47066498  0.26771605
  0.60986504  0.94304656  0.62484327  0.29797895  0.03629869  0.06014885
  0.54961971  0.1587335   0.35129453  0.15526711  0.5084724   0.32063775
  0.45797884  0.02474363  0.07321054  0.48039624  0.63151803  0.87184849
  0.56609319  0.69424001  0.10499014  0.1781253   0.92479686  0.32610236
  0.61655857  0.09921258  0.12527232  0.94503503  0.41045601  0.7149958
  0.31683116  0.94501461  0.74076922  0.12430221  0.18102789  0.14856181
  0.93375234  0.79256664  0.47598451  0.1493183